In [ ]:
#from gameTest import Creature
from creature import *
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx
from random import choice

from qiskit import QuantumRegister, ClassicalRegister, \
    QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.tools.visualization import circuit_drawer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal





varCounter = -1

class Placeholder:
    def __init__(self, id = -1):
        self.id = id
        self.team = False

class AI:
    def __init__(self, creatures):
        clauses = AI.StateToNae(creatures)
        self.clauses = []
        for clause in clauses:
            for threeClause in AI.NaeToNae3(clauses[clause]):
                self.clauses.append(threeClause)

        #print(self.clauses)
        self.vars = []
        for ii in self.clauses:
            for jj in ii:
                if jj[0] not in self.vars:
                    self.vars.append(jj[0])
        print(f'number of variables: {len(self.vars)}')
        self.graph = AI.Nae3ToGraph(self.vars, self.clauses)
        print(self.graph.nodes)


    def draw_graph(self):
        G = self.graph
        default_axes = plt.axes(frameon=True)
        nx.draw_networkx(G, node_color=['r' if (int(node[3:-1]) if node[0] != '-' else int(node[4:-1])) >= 0 else 'b' for node in G.nodes],\
             node_size = 600, alpha = 0.8, ax = default_axes, pos = nx.spring_layout(G))
        
    

        


    def StateToNae(creatures):

        clauses = {ii+kk : [] for ii in ['a','b','c'] for kk in ['True','False']}

        for creature in creatures:
            for gene in ['a', 'b', 'c']:
                if creature.genes[gene] != None:
                    clauses[gene+(str(creature.genes[gene]))] += [creature]

        return clauses


    def NaeToNae3(clause):
        global varCounter
        k = len(clause)
        if k == 0:
            return []
        if k == 1:
            return [[(clause[0], True),(clause[0], True),(clause[0], True)]]
        if k == 2:
            varCounter -= 1
            return [[(clause[0], True), (clause[1], True), (Placeholder(varCounter+1), True)],\
                [(clause[0], False), (clause[1], False), (Placeholder(varCounter+1), True)]]
        if k == 3:
            return [[(c, True) for c in clause]]

        p = [[(clause[0], True), (clause[1], True), (Placeholder(varCounter), True)]]
        for ii in range(2, k-2):
            p.append([(Placeholder(varCounter), False), (clause[ii], True), (Placeholder(varCounter - 1), True)])
            varCounter -= 1
        p.append([(Placeholder(varCounter), False), (clause[-2], True), (clause[-1], True)])
        return p
        
    def Nae3ToGraph(vars, clauses):
        m = len(clauses)
        G = nx.Graph()
        for var in vars:
            G.add_node('X_('+str(var.id)+')')
            G.add_node('-X_('+str(var.id)+')')
        for node in G.nodes:
            if node[0] != '-':
                G.add_edge(node, '-'+node, weight = 10*m)
        for clause in clauses:
            if len(clause) != 3:
                print('failed: nae clause != 3')
                return
            G.add_edge('-'*clause[0][1]+'X_('+str(clause[0][0].id)+')', '-'*clause[1][1]+'X_('+str(clause[1][0].id)+')', weight = 1)
            G.add_edge('-'*clause[0][1]+'X_('+str(clause[0][0].id)+')', '-'*clause[2][1]+'X_('+str(clause[2][0].id)+')', weight = 1)
            G.add_edge('-'*clause[1][1]+'X_('+str(clause[1][0].id)+')', '-'*clause[2][1]+'X_('+str(clause[2][0].id)+')', weight = 1)
        return G

    def bruteForce(weights):
        nodeCount = len(weights)
        bestState = []
        maxCost = 0
        for b in range(2**nodeCount):
            x = [int(t) for t in reversed(list(bin(b)[2:].zfill(nodeCount)))]
            cost = 0
            for i in range(nodeCount):
                for j in range(nodeCount):
                    cost = cost + weights[i, j] * x[i] * (1-x[j])
            if maxCost < cost:
                maxCost = cost
                bestState = x
        return bestState, maxCost

    def geneticAlgorithm():
        pass

    def getWeights(graph):
        nodeToInt = {}
        intToNode = {}

        intCounter = 0

        for ii in graph.nodes:
            nodeToInt[ii] = intCounter
            intCounter += 1
        for ii in nodeToInt:
            intToNode[nodeToInt[ii]] = ii
        nodeCount = len(graph.nodes)
        weights = np.zeros([nodeCount, nodeCount])
        for ii in graph.nodes:
            for jj in graph.nodes:
                temp = graph.get_edge_data(ii, jj, default = 0)
                if temp != 0:
                    weights[nodeToInt[ii], nodeToInt[jj]] = temp['weight']
        return weights

    def optimizeBruteForce(graph):
        bestState, maxCost = AI.bruteForce(AI.getWeights(graph))
        return bestState, maxCost


    def crBruteForce(graph):
        sub_lists = []
        for i in range(0, len(graph.nodes())+1):
            temp = [list(x) for x in combinations(graph.nodes(),i)] 
            sub_lists.extend(temp)

        # cost of all cuts 
        cut_size = []
        for sub_list in sub_lists:
            cut_size.append(
                (sub_list,nx.algorithms.cuts.cut_size(graph,sub_list,eweight='weight'))
            )

        # sort by cost
        return sorted(cut_size, key=lambda item: item[1], reverse=True)


    


    def GraphToHamiltonian():
        pass


def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors,node_size=600,alpha=0.8,ax=default_axes,pos=pos)
    edge_labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels, ax=default_axes)
    edge_labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels, ax=default_axes)

if __name__ == '__main__':
    tf = [True, False]
    creatures = [Creature(True , True , True , True ), Creature(True, True, True, None), Creature(True, True, None, None)]
    ai = AI(creatures)
    colors = ['b' for node in ai.graph.nodes()]
    pos = nx.spring_layout(ai.graph)
    print(ai.graph.nodes)
    draw_graph(ai.graph, colors, pos)
    #plt.show()

    provider = IBMQ.load_account()
    #backend = provider.get_backend('simulator_extended_stabilizer')
    backend = Aer.get_backend("aer_simulator")        
    nodeCount = len(ai.graph.nodes)
    weights = np.zeros([nodeCount, nodeCount])

    nodeToInt = {}
    intToNode = {}

    intCounter = 0

    for ii in ai.graph.nodes:
        nodeToInt[ii] = intCounter
        intCounter += 1
    for ii in nodeToInt:
        intToNode[nodeToInt[ii]] = ii

    print(nodeToInt)
    print(intToNode)

    for ii in ai.graph.nodes:
        for jj in ai.graph.nodes:
            temp = ai.graph.get_edge_data(ii, jj, default = 0)
            if temp != 0:
                weights[nodeToInt[ii], nodeToInt[jj]] = temp['weight']
    
    maxCost = 0
    for b in range(2**nodeCount):
        x = [int(t) for t in reversed(list(bin(b)[2:].zfill(nodeCount)))]
        cost = 0
        for i in range(nodeCount):
            for j in range(nodeCount):
                cost = cost + weights[i, j] * x[i] * (1-x[j])
        if maxCost < cost:
            maxCost = cost
            bestState = x
        
    print(f'\nbest solution: {str(bestState) }. cost = {maxCost}')
    for ii in intToNode:
        print(f'{intToNode[ii]}: {bestState[ii]}')

    print('now testing quantum...')

    maxCut = Maxcut(weights)
    qp = maxCut.to_quadratic_program()
    qubitOp, offset =  qp.to_ising()
    tstr = str(qubitOp).split('\n')
    gates = [ii.split(' ')[-1] for ii in tstr]
    print(gates)
    qReg = QuantumRegister(nodeCount)
    cReg = ClassicalRegister(nodeCount)
    circuit = QuantumCircuit(qReg, cReg)
    for ii in qReg:
        circuit.h(ii)
    for ii in qReg:
        circuit.rx(2, ii)
    for layer in gates:
        qubits = []
        for qubit in range(len(layer)):

            if layer[qubit] == 'Z':
                qubits.append(qubit)
        circuit.rzz(weights[qubits[0], qubits[1]], qReg[qubits[0]], qReg[qubits[1]])
    circuit.measure(qReg, cReg)
    circuit.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'}) 

    #job = execute(circuit, backend, shots = 1000)

    #result = job.result()

    #counts = result.get_counts(circuit)
    #print(counts)
    print('starting quantum')
    quantum_instance = QuantumInstance(backend, seed_simulator=10598, seed_transpiler=10598)
    spsa = SPSA(maxiter=300)
    ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    vqe = VQE(ry, optimizer = spsa, quantum_instance=quantum_instance)
    result = vqe.compute_minimum_eigenvalue(qubitOp)
    x = maxCut.sample_most_likely(result.eigenstate)
    print('energy:', result.eigenvalue.real)
    print('max-cut objective:', result.eigenvalue.real + offset)
    print('solution:', x)
    print('solution objective:', qp.objective.evaluate(x))

    plt.show()
    print(len(gates))

In [ ]:
from numpy.linalg import norm #vector norm
from numpy import array

#
def cost_function(current_state,optimal_state, bot_indices):
    """
    This function takes in a current game state and optimal game state as bitstrings, and finds 
    which move the computer should make (flipping one of the bits). The 'best' move is the bitstring
    that is closest to the optimal state. If there are mutliple optimal states, the first one is picked. 
    """
    #Finding all possible moves
    possible_states = []
    for i in range(len(current_state)):
        if i in bot_indices:
            temp_state = [bit for bit in current_state]
            if current_state[i] == 1:
                temp_state[i] = 0
            elif current_state[i]  == 0:
                temp_state[i] = 1
            else:
                raise Exception('Error: Bits must be 0 or 1')
            print(temp_state)
            possible_states.append(array(temp_state))
    
    #Finding best move 
    #We need to fix this 
    optimal_state = array(optimal_state)
    min = norm(optimal_state - possible_states[0])
    min_state = possible_states[0]
    for state in possible_states:
        if norm(optimal_state - state) < min:
            min = norm(current_state - state)
            min_state = state
    return min_state 

In [ ]:
import pygame, sys
from pygame.locals import *
import time
import itertools
from random import choice, sample
from math import sqrt, ceil
from ai import AI
from cost_function import cost_function

import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import matplotlib.backends.backend_agg as agg
import pylab


from creature import *

#### FOR ME TO ADD
# FENCE
## LOVE MAKING ANIMATION
## PLAYER TURN DISTINCTION
# GRAPH VIS BUTTON

IS_AI_GAME = True


pygame.init()
pygame.font.init()
font = pygame.font.Font(pygame.font.get_default_font(),32)

PLAYER_COLOR = (63+50,0+50,15+50)
COMPUTER_COLOR = (252,90,141)
FONT_COLOR = (240,240,240)
TITLE_FONT_COLOR = FONT_COLOR#(235,235,220)
TITLE_BACKGROUND_COLOR = (100,200,100)
#NAMEPLATE_COLOR = (50,42,50)
NAMEPLATE_COLOR = (0,0,0)
BACKGROUND_COLOR = (100,200,100)

TICK_RATE = 200
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 900

def dist(a, b):
    return sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)


#class Creature:
#
#    __id_iter__ = itertools.count()
#    def __init__(self, team: bool, a = None, b = None, c = None):
#        self.team = team
#        self.group = team
#        self.genes = {'a': a, 'b': b, 'c': c}
#        self.id = next(Creature.__id_iter__)
#        self.age = 0
#
#    def breed(self, other):
#        offspring = Creature(self.team)
#        for ii in self.genes:
#            if self.genes[ii] != None and other.genes[ii] == None:
#                offspring.genes[ii] = self.genes[ii]
#            elif self.genes[ii] == None and other.genes[ii] != None:
#                offspring.genes[ii] = other.genes[ii]
#            elif self.genes[ii] != None and other.genes[ii] != None:
#                offspring.genes[ii] = choice([self.genes[ii], other.genes[ii]])
#        numGenes = {True: 0, False: 0}
#        for ii in offspring.genes:
#            if offspring.genes[ii] != None:
#                numGenes[offspring.genes[ii]] += 1
#        offspring.team = self.team if numGenes[True] == numGenes[False] else \
#            numGenes[True] > numGenes[False]
#        offspring.group = self.group
#        print(f'new creature: {offspring.id}')
#
#        return offspring
#
#    def print(self):
#        print(f'creature id: {self.id}\nteam: {self.team}\ngenes: \
#            {self.genes}\ngroup: {self.group}\nage: {self.age}\n')
#
#    def move(self):
#        self.group = not self.group
#
class Entity:
    def __init__(self, team: bool,size, group = None, posx = 0, posy = 0, genes = (None, None, None)):
        self.creature = Creature(team, a = genes[0], b = genes[1], c = genes[2])
        if group != None:
            self.creature.group = group
        self.posx = posx
        self.posy = posy
        self.lastposx = None
        self.lastposy = None
        self.size = size
        self.fill = PLAYER_COLOR if team else COMPUTER_COLOR
        aText = 'T' if genes[0] else 'N' if genes[0] == None else 'F'
        bText = 'T' if genes[1] else 'N' if genes[1] == None else 'F'
        cText = 'T' if genes[2] else 'N' if genes[2] == None else 'F'
        self.idLabel = font.render(aText+bText+cText, True, FONT_COLOR, NAMEPLATE_COLOR)
        # image credit: https://www.pngkey.com/png/full/822-8224774_cow-clipart-vector-cow-cartoon-images-transparent.png
        self.imageSprite = pygame.image.load('cow.png').convert_alpha() # dimensions 640x480 in original
        scale = 7
        self.imageDimension = (640/scale, 480/scale)
        self.imageSprite = pygame.transform.scale(self.imageSprite, self.imageDimension)
        self.imageSprite.fill(self.fill, special_flags=pygame.BLEND_ADD) # has to be duplicated, hackathon lol

    def makeFromCreature(self, creature, size, posx = 0, posy = 0):
        self.creature = creature
        self.posx = posx
        self.posy = posy
        self.size = size
        aText = 'T' if creature.genes['a'] else 'N' if creature.genes['a'] == None else 'F'
        bText = 'T' if creature.genes['b'] else 'N' if creature.genes['b'] == None else 'F'
        cText = 'T' if creature.genes['c'] else 'N' if creature.genes['c'] == None else 'F'
        self.fill = PLAYER_COLOR if creature.team else COMPUTER_COLOR
        self.idLabel = font.render(aText+bText+cText, True, (0, 255, 0), (0, 0, 255))
        self.imageSprite = pygame.image.load('cow.png').convert_alpha() # dimensions 640x480 in original
        scale = 7
        self.imageDimension = (640/scale, 480/scale)
        self.imageSprite = pygame.transform.scale(self.imageSprite, self.imageDimension)
        self.imageSprite.fill(self.fill, special_flags=pygame.BLEND_ADD) # has to be duplicated, hackathon lol

    def setPos(self, posx = 0, posy = 0):
        self.lastposx = self.posx
        self.lastposy = self.posy
        self.posx = posx
        self.posy = posy

    def getPos(self):
        return self.posx,self.posy

    def getPosRange(self):
        # solve for the line between the two positions
        inverseSpeed = 10
        # if infinite slope
        if self.posx == self.lastposx:
            print('DEBUG: no x movement, avoided "div by zero"')
            positions = [(self.lastposx,self.lastposy)]
            totalYDist = self.posy - self.lastposy
            for i in range(inverseSpeed):
                positions.append((self.lastposx, positions[-1][1]+totalYDist/inverseSpeed))
        # get length of the line, subdivide
        else:
            slope = (self.posy-self.lastposy)/(self.posx-self.lastposx)
            yIntercept = self.posy - slope*self.posx
            positions = [(self.lastposx, self.lastposy)]
            totalXDist = self.posx-self.lastposx
            for i in range(inverseSpeed):
                nextX = positions[-1][0] + totalXDist/inverseSpeed
                positions.append((nextX, slope*nextX + yIntercept))
        return positions

class Manager:
    def __init__(self, screen, screenWidth, screenHeight, maxRowSize = 5):
        self.clock = pygame.time.Clock() # variables for running the game
        self.heart = pygame.image.load('heart.png') # 320x290 https://www.freeiconspng.com/images/heart-png
        scale = 8
        self.heartImageDimension = (320/scale, 290/scale)
        self.heart = pygame.transform.scale(self.heart, self.heartImageDimension)

        self.skullAndBones = pygame.image.load('skull-and-crossbones.png') # 320x308 https://www.freeiconspng.com/images/skull-and-crossbones-png
        scale = 4
        self.skullAndBonesImageDimension = (320/scale, 308/scale)
        self.skullAndBones = pygame.transform.scale(self.skullAndBones, self.skullAndBonesImageDimension)

        self.titleFont = pygame.font.Font(pygame.font.get_default_font(),32)
        self.phaseMessage = self.titleFont.render('', True, FONT_COLOR, BACKGROUND_COLOR)

        self.graphButtonMessage = self.titleFont.render('Display MaxCut Graph', True, NAMEPLATE_COLOR, FONT_COLOR)
        self.graphButton = self.graphButtonMessage.get_rect()
        self.graphButton.center = (SCREEN_WIDTH/2, SCREEN_HEIGHT-75)#-SCREEN_HEIGHT*.95)

        self.entities = []
        self.screen = screen
        self.maxRowSize = maxRowSize
        self.screenWidth = screenWidth
        self.screenHeight = screenHeight
        self.numEntities = 0
        self.entitySize = screenWidth/(2*maxRowSize) - screenWidth/30

        self.drawGraph = False

    def addEntity(self, team, group, genes = (None, None, None)):
        self.entities.append(Entity(team, self.entitySize, group=group, genes = genes))
        self.numEntities += 1
        self.arrangeEntities()

    def addEntityFromCreature(self, creature):
        newEntity = Entity(False, 0)
        newEntity.makeFromCreature(creature, self.entitySize)
        self.entities.append(newEntity)
        self.numEntities += 1
        self.arrangeEntities()

    def drawEntities(self):

        for entity in self.entities:
            entitySurface = pygame.Surface((self.entitySize*2, self.entitySize*2))
            #entitySurface.fill((255,255,255))
            entitySurface.fill(BACKGROUND_COLOR)
            entitySurface.set_alpha(50)
            #pygame.draw.circle(entitySurface, tuple(entity.fill), (self.entitySize, self.entitySize), entity.size)
            #pygame.draw.circle(self.screen, tuple(entity.fill) if entity.creature.age > 0 else (247, 255, 0), (entity.posx, entity.posy), self.entitySize, ceil(self.entitySize/10))
            pygame.draw.circle(self.screen, tuple(entity.fill) , (entity.posx, entity.posy), self.entitySize, ceil(self.entitySize/10))


            # nameplate
            #tr = entity.idLabel.get_rect()
            #tr.center = (entity.posx, entity.posy+entity.imageDimension[1])
            pygame.draw.rect(self.screen, NAMEPLATE_COLOR, pygame.Rect(entity.posx-30, entity.posy+entity.imageDimension[1]/3*2, 60, 30))
            pygame.draw.rect(self.screen, FONT_COLOR, pygame.Rect(entity.posx-27, entity.posy+entity.imageDimension[1]/3*2+3, 54, 24))

            # fill colors for genes
            pygame.draw.rect(self.screen, PLAYER_COLOR if entity.creature.genes['a'] else FONT_COLOR if entity.creature.genes['a'] == None else COMPUTER_COLOR, 
                             pygame.Rect(entity.posx-27, entity.posy+entity.imageDimension[1]/3*2+3, 54/3,24))
            pygame.draw.rect(self.screen, PLAYER_COLOR if entity.creature.genes['b'] else FONT_COLOR if entity.creature.genes['b'] == None else COMPUTER_COLOR, 
                             pygame.Rect(entity.posx-27+54/3, entity.posy+entity.imageDimension[1]/3*2+3, 54/3,24))
            pygame.draw.rect(self.screen, PLAYER_COLOR if entity.creature.genes['c'] else FONT_COLOR if entity.creature.genes['c'] == None else COMPUTER_COLOR, 
                             pygame.Rect(entity.posx-27+54/3*2, entity.posy+entity.imageDimension[1]/3*2+3, 54/3,24))

            # borders for nameplate
            pygame.draw.rect(self.screen, NAMEPLATE_COLOR, pygame.Rect(entity.posx-27+54/3, entity.posy+entity.imageDimension[1]/3*2, 3, 30))
            pygame.draw.rect(self.screen, NAMEPLATE_COLOR, pygame.Rect(entity.posx-27+54/3*2, entity.posy+entity.imageDimension[1]/3*2, 3, 30))

            self.screen.blit(entity.imageSprite, (entity.posx-entity.imageDimension[0]/2, entity.posy-entity.imageDimension[1]/2))
            self.screen.blit(entitySurface, (entity.posx-self.entitySize,entity.posy-self.entitySize))
            #self.screen.blit(entity.idLabel, tr)


        # phase of game
        tr = self.phaseMessage.get_rect()
        tr.center = (SCREEN_WIDTH/2, SCREEN_HEIGHT-150)#-SCREEN_HEIGHT*.95)
        self.screen.blit(self.phaseMessage, tr)

        # graph message
        self.screen.blit(self.graphButtonMessage, self.graphButton)

        if self.drawGraph:
            self.graphImage = pygame.image.load("graph.png")
            self.screen.blit(self.graphImage, (SCREEN_WIDTH+50, 100))
            #self.drawImage(self.graphImage, (SCREEN_WIDTH+100, 0))
    
    def getArrangement(self):
        arrangement = []
        trueEntities = []
        falseEntities = []
        for ii in self.entities:
            if ii.creature.group:
                trueEntities.append(ii)
            else:
                falseEntities.append(ii)

        numTrue = len(trueEntities)
        numFalse = len(falseEntities)
        
        trueRows = [0]
        currentCount = 0
        currentRow = 0
        while(numTrue):
            if currentCount == 5:
                currentCount = 0
                currentRow += 1
                trueRows.append(0)
            numTrue -= 1
            currentCount += 1
            trueRows[currentRow] += 1
        falseRows = [0]
        currentCount = 0
        currentRow = 0
        while(numFalse):
            if currentCount == 5:
                currentCount = 0
                currentRow += 1
                falseRows.append(0)
            numFalse -= 1
            currentCount += 1
            falseRows[currentRow] += 1
        



        for ii in range(len(falseRows)):
            for jj in range(falseRows[ii]):
                arrangement.append([self.screenWidth/falseRows[ii]*(jj+0.5), self.screenWidth/self.maxRowSize * (ii+1/2), False])
       
        for ii in range(len(trueRows)):
            for jj in range(trueRows[ii]):
                arrangement.append([self.screenWidth/trueRows[ii]*(jj+0.5), self.screenWidth-self.screenWidth/self.maxRowSize * (ii+1/2), True])
         
        return arrangement

    def arrangeEntities(self):
        arrangement = self.getArrangement()
        newPositions = [None] * self.numEntities
        for ii in arrangement:
            minDistance = 10000000000
            best = None
            for kk in range(self.numEntities):
                if dist((self.entities[kk].posx, self.entities[kk].posy), (ii[0], ii[1])) < minDistance \
                    and newPositions[kk] == None and self.entities[kk].creature.group == ii[2]:
                    best = kk
                    minDistance = dist((self.entities[kk].posx, self.entities[kk].posy), (ii[0], ii[1]))
            newPositions[best] = ii

        for ii in range(self.numEntities):
            currPos = self.entities[ii].getPos()
            if currPos == (newPositions[ii][0], newPositions[ii][1]):
                continue
            self.entities[ii].setPos(newPositions[ii][0], newPositions[ii][1])
            # implement animation; for now, just slide cows across the screen
            posRange = self.entities[ii].getPosRange()
            for pos in posRange:
                self.entities[ii].setPos(pos[0],pos[1])
                #self.refreshScreen()
                self.drawScreen()
                self.clock.tick(TICK_RATE)

            #if currPos != (newPositions[ii][0], newPositions[ii][1]):
            #    print(currPos)
            #    print((newPositions[ii][0], newPositions[ii][1]))
            #    print('blob moved!')
            #    exit(1)

    def breed(self):

        self.phaseMessage= self.titleFont.render('Breed Phase', True, TITLE_FONT_COLOR, TITLE_BACKGROUND_COLOR)

        trueEntities = []
        falseEntities = []
        for ii in self.entities:
            if ii.creature.group:
                trueEntities.append(ii)
            else:
                falseEntities.append(ii)
        trueBreeders = []
        falseBreeders = []
        if len(trueEntities) >= 2:
            trueBreeders = sample(trueEntities, 2)
        if len(falseEntities) >= 2:
            falseBreeders = sample(falseEntities, 2)
        if trueBreeders:
            # do breeding animation
            self.animateBreed(True, trueBreeders[0], trueBreeders[1])
            self.addEntityFromCreature(Creature.breed(trueBreeders[0].creature, \
                trueBreeders[1].creature))
        if falseBreeders:
            self.animateBreed(False, falseBreeders[0], falseBreeders[1])
            self.addEntityFromCreature(Creature.breed(falseBreeders[0].creature, \
                falseBreeders[1].creature))
        self.arrangeEntities()

    def animateBreed(self, group, creature1, creature2):
        print(creature1, creature2)
        for i,creature in enumerate([creature1, creature2]):
            destx = 300 + i*50
            if group:
                desty = 360
            else:
                desty = 240
            #desty = 460 if group else 440
            creature.setPos(destx, desty)
            posRange = creature.getPosRange()
            for pos in posRange:
                creature.setPos(pos[0],pos[1])
                self.drawScreen()
                self.clock.tick(TICK_RATE)
        self.drawImage(self.heart, (creature1.posx, creature1.posy-50))
        time.sleep(2)

    def kill(self):
        
        self.phaseMessage = self.titleFont.render('Death Phase', True, TITLE_FONT_COLOR, TITLE_BACKGROUND_COLOR)

        trueEntities = []
        falseEntities = []
        for ii in range(len(self.entities)):
            if self.entities[ii].creature.group:
                trueEntities.append(ii)
            else:
                falseEntities.append(ii)
        trueWeightedElements = []
        falseWeightedElements = []
        for ii in trueEntities:
            trueWeightedElements.extend([ii]*(self.entities[ii].creature.age))
        for ii in falseEntities:
            falseWeightedElements.extend([ii]*(self.entities[ii].creature.age))
        tInd = None
        if trueWeightedElements: 
            ind = choice(trueWeightedElements)
            tInd = ind
            print(self.entities[ind].fill)
            #self.entities[ind].fill[1] = 255
            print(self.entities[ind].fill)
            #self.refreshScreen()

            self.drawImage(self.skullAndBones, (self.entities[ind].posx-self.skullAndBonesImageDimension[0]/2, self.entities[ind].posy-self.skullAndBonesImageDimension[0]/2))

            time.sleep(2)

            #self.clock.tick(600)
            print(f'killed true creature: {self.entities[ind].creature.id}\nGroup: \
                {self.entities[ind].creature.group}\nteam: {self.entities[ind].creature.team}')
            del self.entities[ind]
            self.numEntities -= 1


        if falseWeightedElements:
            ind = choice(falseWeightedElements)
            if tInd != None and ind > tInd:
                ind -= 1
            #self.entities[ind].fill[1] = 255
            #self.refreshScreen()

            self.drawImage(self.skullAndBones, (self.entities[ind].posx-self.skullAndBonesImageDimension[0]/2, self.entities[ind].posy-self.skullAndBonesImageDimension[0]/2))
            time.sleep(2)
            #self.clock.tick(600)
            print(f'killed false creature: {self.entities[ind].creature.id}\nGroup: \
                {self.entities[ind].creature.group}\nteam: {self.entities[ind].creature.team}')

            del self.entities[ind]
            self.numEntities -= 1
    
    def age(self):
        for ii in self.entities:
            ii.creature.age += 1
            ii.fill = [kk - 10 * ii.creature.age \
                if kk - 10 * ii.creature.age >= 0 else 0 \
                    for kk in ii.fill]
            print(ii.fill)

    def refreshScreen(self):
        self.screen.fill(BACKGROUND_COLOR)
        self.arrangeEntities()
        self.drawEntities()
        pygame.display.flip()

    def drawScreen(self):
        self.screen.fill(BACKGROUND_COLOR)
        self.drawEntities()
        pygame.display.flip()       

    def drawImage(self, image, pos):
        self.screen.fill(BACKGROUND_COLOR)
        self.drawEntities()
        self.screen.blit(image, pos)#(self.entities[ind].posx, self.entities[ind].posy))
        pygame.display.flip()       

    def saveMaxCutGraph(self):
        ai = AI([entity.creature for entity in self.entities])
        #ai.draw_graph()
        G = ai.graph
        default_axes = plt.axes(frameon=True)
        nx.draw_networkx(G, node_color=['r' if (int(node[3:-1]) if node[0] != '-' else int(node[4:-1])) >= 0 else 'b' for node in G.nodes],\
             node_size = 600, alpha = 0.8, ax = default_axes, pos = nx.spring_layout(G))
        plt.savefig("graph.png",format="PNG")

        #self.graphImage = pygame.image.load("graph.png")
        #self.drawImage(self.graphImage, (SCREEN_WIDTH+100, 0))
        #fig = pylab.figure(figsize=[4,4], dpi=100) 
        ##ax = fig.gca()
        #canvas = agg.FigureCanvasAgg(fig)
        #canvas.draw()
        #renderer = canvas.get_renderer()
        #raw_data = renderer.tostring_rgb()

        #image = pygame.image.fromstring(raw_data)#,  "RGB")
        #self.screen.blit(image, (0,0))
        #self.drawScreen()


    def getMove(self):
        creatures = [entity.creature for entity in self.entities]
        ai = AI(creatures)

        nodeToInt = {}
        intToNode = {}
        intCounter = 0

        for ii in ai.graph.nodes:
            nodeToInt[ii] = intCounter
            intCounter += 1
        for ii in nodeToInt:
            intToNode[nodeToInt[ii]] = ii
        optimalState, optimalScore = AI.optimizeBruteForce(ai.graph)
        currentState = [int(creature.group) for creature in creatures]
        orderedOptimalState = []
        botIndices = []
        for creature in creatures:
            print(nodeToInt)
            print(entity.creature.id for entity in self.entities)
            orderedOptimalState.append(optimalState[nodeToInt['X_('+str(creature.id)+')']])
        for ii in range(len(creatures)):
            if not creatures[ii].team:
                botIndices.append(ii)
        newState = cost_function(currentState, orderedOptimalState, botIndices)
        print('Current State:',currentState, 'New State:', newState)
        for ii in range(len(newState)):
            if newState[ii] != currentState[ii]:
                return creatures[ii]
        print('something went wrong in getMove')
        return None

    def endCheck(self):
        color = self.entities[0].creature.team
        for entity in self.entities:
            if entity.creature.team != color:
                return False
        return True 
                
      
        

def main():

    screenWidth = SCREEN_WIDTH
    screenHeight = SCREEN_HEIGHT

    #pygame.init()
    screen = pygame.display.set_mode((screenWidth, screenHeight))#, DOUBLEBUF)

    manager = Manager(screen, screenWidth, screenHeight)

    #menu()
    # MENU
    entitySurface = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
    entitySurface.fill(BACKGROUND_COLOR)
    screen.blit(entitySurface, (0,0))

    titleCardFont = pygame.font.Font(pygame.font.get_default_font(),70)
    title = titleCardFont.render('MILQ Simulator', True, FONT_COLOR, BACKGROUND_COLOR)
    titleCard = title.get_rect()
    titleCard.center = (SCREEN_WIDTH/2, 200)
    screen.blit(title, titleCard)

    menuFont = pygame.font.Font(pygame.font.get_default_font(),32)
    menu4Message= menuFont.render('Easy, small to simulate', True, PLAYER_COLOR, FONT_COLOR)
    menu6Message= menuFont.render('Medium, computer beware', True, PLAYER_COLOR, FONT_COLOR)
    menu10Message = menuFont.render('Hard, wait for heat death', True, PLAYER_COLOR, FONT_COLOR)

    button4 = menu4Message.get_rect()
    button4.center = (SCREEN_WIDTH/2, 400)#-SCREEN_HEIGHT*.95)
    button6 = menu6Message.get_rect()
    button6.center = (SCREEN_WIDTH/2, 500)#-SCREEN_HEIGHT*.95)
    button10 = menu10Message.get_rect()
    button10.center = (SCREEN_WIDTH/2, 600)#
    screen.blit(menu4Message, button4)
    screen.blit(menu6Message, button6)
    screen.blit(menu10Message, button10)

    pygame.display.flip()
    #time.sleep(5)

    # START IN MENU
    difficulty = None
    while difficulty is None:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: sys.exit()
            if event.type == pygame.MOUSEBUTTONUP:
                pos = pygame.mouse.get_pos()
                if pos[0] >= button4.left and pos[0] <= button4.right and pos[1] >= button4.top and pos[1] <= button4.bottom:
                    difficulty = 4
                    manager.addEntity(True, group=True, genes = (True, True, True))
                    manager.addEntity(True, group=True, genes = (True, True, None))
                    manager.addEntity(False, group=False, genes = (False, False, False))
                    manager.addEntity(False, group=False, genes = (False, False, None))
                elif pos[0] >= button6.left and pos[0] <= button6.right and pos[1] >= button6.top and pos[1] <= button6.bottom:
                    difficulty = 6
                    manager.addEntity(True, group=True, genes = (True, True, True))
                    manager.addEntity(True, group=True, genes = (True, True, None))
                    manager.addEntity(True, group=True, genes = (True, None, None))
                    manager.addEntity(False, group=False, genes = (False, False, False))
                    manager.addEntity(False, group=False, genes = (False, False, None))
                    manager.addEntity(False, group=False, genes = (False, None, None))
                elif pos[0] >= button10.left and pos[0] <= button10.right and pos[1] >= button10.top and pos[1] <= button10.bottom:
                    difficulty = 10
                    manager.addEntity(True, group=True, genes = (True, True, True))
                    manager.addEntity(True, group=True, genes = (True, True, None))
                    manager.addEntity(True, group=True, genes = (True, True, None))
                    manager.addEntity(True, group=True, genes = (True, None, None))
                    manager.addEntity(True, group=True, genes = (True, None, None))
                    manager.addEntity(False, group=False, genes = (False, False, False))
                    manager.addEntity(False, group=False, genes = (False, False, None))
                    manager.addEntity(False, group=False, genes = (False, False, None))
                    manager.addEntity(False, group=False, genes = (False, None, None))
                    manager.addEntity(False, group=False, genes = (False, None, None))
    #if IS_AI_GAME:
    #    manager.addEntity(True, group=True, genes = (True, True, True))
    #    manager.addEntity(True, group=True, genes = (True, True, None))
    #    manager.addEntity(False, group=False, genes = (False, False, False))
    #    manager.addEntity(False, group=False, genes = (False, False, None))

    #else:
    #    manager.addEntity(True, group=True, genes = (True, True, True))
    #    manager.addEntity(True, group=True, genes = (True, True, None))
    #    manager.addEntity(True, group=True, genes = (True, True, None))
    #    manager.addEntity(True, group=True, genes = (True, None, None))
    #    manager.addEntity(True, group=True, genes = (True, None, None))
    #    manager.addEntity(False, group=False, genes = (False, False, False))
    #    manager.addEntity(False, group=False, genes = (False, False, None))
    #    manager.addEntity(False, group=False, genes = (False, False, None))
    #    manager.addEntity(False, group=False, genes = (False, None, None))
    #    manager.addEntity(False, group=False, genes = (False, None, None))
    manager.age()



    maxRowSize = 10
    blobGap = screenWidth/60

    blobWidth = screenWidth/maxRowSize - (2*blobGap)






    clicked = None
    activeCreature = None
    currentTurn = True
    moveCounter = False

    #displayGraph = False

    while(True):

        manager.clock.tick(3) 

        for event in pygame.event.get():

            if currentTurn:
                manager.phaseMessage= manager.titleFont.render('Brown to Moove', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
            else:
                manager.phaseMessage= manager.titleFont.render('Pink to Moove', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)

            if event.type == pygame.QUIT: sys.exit()


            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                for ii in manager.entities:
                    if sqrt((ii.posx-pos[0])**2+(ii.posy-pos[1])**2) < blobWidth/2+blobGap:
                        clicked = ii

            if event.type == pygame.MOUSEBUTTONUP:
                pos = pygame.mouse.get_pos()
                # if display graph clicked
                graphButton = manager.graphButton
                if pos[0] >= graphButton.left and pos[0] <= graphButton.right and pos[1] >= graphButton.top and pos[1] <= graphButton.bottom:
                    #displayGraph = not displayGraph
                    manager.drawGraph = not manager.drawGraph
                    screen = pygame.display.set_mode((screenWidth + 5/4*screenWidth*int(manager.drawGraph), screenHeight))#, DOUBLEBUF)
                    manager.screen = screen
                
                if manager.drawGraph:
                    manager.saveMaxCutGraph()

                # if cow clicked
                for ii in manager.entities:
                    if sqrt((ii.posx-pos[0])**2+(ii.posy-pos[1])**2) < blobWidth/2+blobGap:
                        if ii == clicked:
                            print(clicked.creature.team)
                            if ii.creature.team == currentTurn:
                                ii.creature.move()
                                currentTurn = not currentTurn
                                if currentTurn:
                                    manager.phaseMessage= manager.titleFont.render('Brown to Moove', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                elif IS_AI_GAME:
                                    manager.phaseMessage= manager.titleFont.render('Quantum Cows colluding...', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                else:
                                    manager.phaseMessage= manager.titleFont.render('Pink to Moove', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                manager.refreshScreen()
                                time.sleep(2)
                                if not currentTurn and IS_AI_GAME:
                                    if manager.drawGraph:
                                        manager.saveMaxCutGraph()
                                    #manager.phaseMessage= manager.titleFont.render('Pink to Moove', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                    print('Bot is Thinking')
                                    manager.getMove().move()
                                    manager.refreshScreen()
                                    time.sleep(2)
                                    currentTurn = not currentTurn
                                    moveCounter = not moveCounter
                                if moveCounter:
                                    manager.age()
                                    manager.breed()
                                    manager.kill()
                                    if(manager.endCheck()):
                                        print("End game")
                                        if(manager.entities[0].creature.team):
                                            manager.phaseMessage= manager.titleFont.render('You Win!!', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                        else:
                                            manager.phaseMessage= manager.titleFont.render('You lose :(', True, TITLE_FONT_COLOR, BACKGROUND_COLOR)
                                        manager.refreshScreen()
                                        time.sleep(15)
                                        return 
                                    
                                    
                                moveCounter = not moveCounter
                clicked = None


        manager.refreshScreen()
        #for ii in range(len(trueArrangement)):
        #    pygame.draw.circle(screen, (0, 0, 255), (trueArrangement[ii][0], trueArrangement[ii][1]), blobWidth/2)
        #   pygame.draw.circle(screen, (255, 0, 0), (falseArrangement[ii][0], falseArrangement[ii][1]), blobWidth/2)
        pygame.display.flip()


    c1 = Creature(False, a = False)
    c2 = Creature(False)
    c3 = Creature(True, a = True, b = True)
    c4 = Creature(True)
    
    babies = [Creature.breed(c1, c3) for ii in range(5)]
    for ii in babies:
        ii.print()


if __name__ == '__main__':
    main()
